In [26]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

Run one of the cells reading dataset below. Model will adjust automatically.

In [63]:
news = pd.read_csv('News.csv')
y = news.category
X = news.drop(['category'], axis=1)
news.head()

,headline,category,authors,short_description
0,There Were 2 Mass Shootings In Texas Last Wee...,CRIME,Melissa Jeltsen,She left her husband. He killed their childre...
1,Will Smith Joins Diplo And Nicky Jam For The ...,ENTERTAINMENT,Andy McDonald,Of course it has a song.
2,Hugh Grant Marries For The First Time At Age ...,ENTERTAINMENT,Ron Dicker,The actor and his longtime girlfriend Anna Eb...
3,Jim Carrey Blasts 'Castrato' Adam Schiff And ...,ENTERTAINMENT,Ron Dicker,The actor gives Dems an ass-kicking for not f...
4,Julianna Margulies Uses Donald Trump Poop Bag...,ENTERTAINMENT,Ron Dicker,The Dietland actress said using the bags is a...


In [29]:
tortilla = pd.read_csv('tortilla_prices.csv')
y = tortilla['Store type']
X = tortilla.drop(['Store type'], axis=1)
tortilla.head()

,State,City,Year,Month,Day,Store type,Price per kilogram
0,Aguascalientes,Aguascalientes,2007,1,10,Mom and Pop Store,9.9
1,Baja California,Mexicali,2007,1,10,Mom and Pop Store,NaN
2,Baja California,Tijuana,2007,1,10,Mom and Pop Store,10.0
3,Baja California Sur,La Paz,2007,1,10,Mom and Pop Store,10.0
4,Campeche,Campeche,2007,1,10,Mom and Pop Store,10.0


In [30]:
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and  X[cname].dtype == "object"]
vectorized_cols = [cname for cname in X.columns if X[cname].nunique() >= 10 and  X[cname].dtype == "object"]
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureMerger(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  
  def transform(self, X):
    df = pd.DataFrame()
    df['result'] = X.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    return df['result']

In [32]:
numerical_transformer = SimpleImputer(strategy='median')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

vectorized_transformer = Pipeline(steps=[
  ('merge', FeatureMerger()),
  ('vectorize', TfidfVectorizer(max_features=2000))
]) 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('vect', vectorized_transformer, vectorized_cols)
    ])

In [33]:
# TODO: try another model
model = LogisticRegression(max_iter=1000)

In [34]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

accuracy = cross_val_score(my_pipeline, X, y, cv=5, scoring='accuracy')

print("Average accuracy:", accuracy.mean())

Average accuracy: 0.7084084947764306
